# Part 2: Training

In this part we will show how using MLRun's **Feature Store** we can easily define a **Feature Vector** and create the dataset we need to run our training process.  
By the end of this tutorial you’ll learn how to:
- Combine multiple data sources to a single Feature Vector
- Create training dataset
- Create a model using an MLRun Hub function

In [1]:
project_name = 'fraud-demo'

In [2]:
import mlrun

# Initialize the MLRun project object
project = mlrun.get_or_create_project(project_name, context="./", user_project=True)

> 2023-02-09 13:24:31,015 [info] loaded project fraud-demo from MLRun DB


## Step 1 - Create a Feature Vector  
In this section we will create our Feature Vector.  
The Feature vector will have a `name` so we can reference to it later via the URI or our serving function, and a list of `features` from the available FeatureSets.  We can add a feature from a feature set by adding `<FeatureSet>.<Feature>` to the list, or add `<FeatureSet>.*` to add all the FeatureSet's available features.  

By default, the first FeatureSet in the feature list will act as the spine. meaning that all the other features will be joined to it.  
For example, in this instance we use the early sense sensor data as our spine, so for each early sense event we will create produce a row in the resulted Feature Vector.

In [3]:
# Define the list of features we will be using
features = ['events.*',
            'transactions.amount_max_2h', 
            'transactions.amount_min_2h', 
            'transactions.amount_avg_2h', 
            'transactions.es_transportation_sum_1d', 
            'transactions.es_health_sum_1d',
            'transactions.es_otherservices_sum_1d', 
            'transactions.es_food_sum_1d',
            'transactions.es_hotelservices_sum_1d', 
            'transactions.es_barsandrestaurants_sum_1d',
            'transactions.es_tech_sum_1d', 
            'transactions.es_sportsandtoys_sum_1d',
            'transactions.es_wellnessandbeauty_sum_1d', 
            'transactions.es_hyper_sum_1d',
            'transactions.es_fashion_sum_1d', 
            'transactions.es_home_sum_1d', 
            'transactions.es_contents_sum_1d', 
            'transactions.es_travel_sum_1d', 
            'transactions.es_leisure_sum_1d',
            'transactions.gender_F',
            'transactions.gender_M',
            'transactions.step', 
            'transactions.amount', 
            'transactions.timestamp_hour',
            'transactions.timestamp_month',
            ]

In [4]:
# Import MLRun's Feature Store
import mlrun.feature_store as fstore

# Define the feature vector name for future reference
fv_name = 'transactions-fraud'

# Define the feature vector using our Feature Store (fstore)
transactions_fv = fstore.FeatureVector(fv_name,
                          features, 
                          label_feature="labels.label",
                          description='Predicting a fraudulent transaction')

# Save the feature vector in the Feature Store
transactions_fv.save()

## Step 2 - Preview the Feature Vector Data

Obtain the values of the features in the feature vector, to ensure the data appears as expected

In [5]:
# Import the Parquet Target so we can directly save our dataset as a file
from mlrun.datastore.targets import ParquetTarget

# Get offline feature vector as dataframe and save the dataset to parquet
train_dataset = fstore.get_offline_features(fv_name, target=ParquetTarget())

> 2023-02-09 13:24:32,796 [info] wrote target: {'name': 'parquet', 'kind': 'parquet', 'path': 'v3io:///projects/fraud-demo-dani/FeatureStore/transactions-fraud/parquet/vectors/transactions-fraud-latest.parquet', 'status': 'ready', 'updated': '2023-02-09T13:24:32.796165+00:00', 'size': 225670, 'partitioned': True}


In [6]:
# Preview our dataset
train_dataset.to_dataframe().tail(5)

,event_password_change,event_details_change,event_login,amount_max_2h,amount_min_2h,amount_avg_2h,es_transportation_sum_1d,es_health_sum_1d,es_otherservices_sum_1d,es_food_sum_1d,...,es_contents_sum_1d,es_travel_sum_1d,es_leisure_sum_1d,gender_F,gender_M,step,amount,timestamp_hour,timestamp_month,label
8562,1,0,0,22.35,1.71,12.560000,41.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,101.0,13.62,12,02,0.0
8563,1,0,0,51.57,24.55,35.003333,30.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,136.0,28.89,12,02,0.0
8564,1,0,0,44.37,5.63,21.732000,36.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,40.0,12.82,12,02,0.0
8565,0,0,1,38.21,13.95,24.466667,22.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,117.0,38.21,12,02,0.0
8566,0,0,1,56.04,6.64,37.222500,31.0,0.0,0.0,4.0,...,0.0,0.0,0.0,1.0,0.0,17.0,34.10,12,02,0.0


## Step 3 - Train Models and Choose Highest Accuracy

With MLRun, one can easily train different models and compare the results. In the code below, we train 3 different models,
each uses a different algorithm (random forest, XGBoost, adabost), and choose the model with the highest accuracy

In [7]:
# Import the Sklearn classifier function from the functions hub
classifier_fn = mlrun.import_function('hub://auto_trainer')

In [8]:
# Prepare the parameters list for the training function
# We will be using 3 different models
training_params = {"model_name": ['transaction_fraud_rf', 
                                  'transaction_fraud_xgboost', 
                                  'transaction_fraud_adaboost'],
              
                   "model_class": ['sklearn.ensemble.RandomForestClassifier',
                                  'sklearn.ensemble.GradientBoostingClassifier',
                                  'sklearn.ensemble.AdaBoostClassifier'],
                   "tag": 'latest'}

# Define the training task, including our feature vector, label and hyperparams definitions
train_task = mlrun.new_task('training', 
                      inputs={'dataset': transactions_fv.uri},
                      params={'label_columns': 'label'}
                     )

train_task.with_hyper_params(training_params, strategy='list', selector='max.accuracy')

# Specify our cluster image
classifier_fn.spec.image = 'mlrun/mlrun'

# Run training
classifier_fn.run(train_task, local=False)

> 2023-02-09 13:24:33,451 [info] starting run training uid=78476c0f382e4f1c9dfe89134e41dd34 DB=http://mlrun-api:8080
> 2023-02-09 13:24:33,657 [info] Job is running in the background, pod: training-5bs9z
> 2023-02-09 13:24:38,859 [info] test_set or train_test_split_size are not provided, setting train_test_split_size to 0.2
> 2023-02-09 13:24:40,813 [info] label columns: label
> 2023-02-09 13:24:40,814 [info] Sample set not given, using the whole training set as the sample set
> 2023-02-09 13:24:41,085 [info] training 'transaction_fraud_rf'
> 2023-02-09 13:24:44,316 [info] test_set or train_test_split_size are not provided, setting train_test_split_size to 0.2
> 2023-02-09 13:24:46,161 [info] label columns: label
> 2023-02-09 13:24:46,161 [info] Sample set not given, using the whole training set as the sample set
> 2023-02-09 13:24:46,166 [info] training 'transaction_fraud_xgboost'
> 2023-02-09 13:24:50,375 [info] test_set or train_test_split_size are not provided, setting train_test_s

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
fraud-demo-dani,...4e41dd34,0,Feb 09 13:24:38,completed,training,v3io_user=danikind=jobowner=danimlrun/client_version=1.3.0-rc20mlrun/client_python_version=3.9.13,dataset,label_columns=label,best_iteration=1accuracy=0.9959159859976663f1_score=0.7586206896551724precision_score=0.8461538461538461recall_score=0.6875,feature-importancetest_setconfusion-matrixroc-curvescalibration-curvemodeliteration_resultsparallel_coordinates


> 2023-02-09 13:24:58,305 [info] run executed, status=completed


## Step 4 - Perform Feature Selection

As part of our data science process we will try and reduce the training dataset's size to get rid of bad or unuseful features and save computation time.

We will use our ready-made feature selection function from our hub [`hub://feature_selection`](https://github.com/mlrun/functions/blob/development/feature_selection/feature_selection.ipynb) to select the best features to keep on a sample from our dataset and run the function on that.


In [9]:
feature_selection_fn = mlrun.import_function('hub://feature_selection')

feature_selection_run = feature_selection_fn.run(
            params={"k": 18,
                    "min_votes": 2,
                    "label_column": 'label',
                    'output_vector_name':fv_name + "-short",
                    'ignore_type_errors': True},
    
            inputs={'df_artifact': transactions_fv.uri},
            name='feature_extraction',
            handler='feature_selection',
    local=False)

> 2023-02-09 13:24:58,688 [info] starting run feature_extraction uid=226a485cf9df45f9b4fff8c978a7bdf1 DB=http://mlrun-api:8080
> 2023-02-09 13:24:58,857 [info] Job is running in the background, pod: feature-extraction-lpctf
> 2023-02-09 13:25:04,544 [info] Couldn't calculate f_classif because of: bad operand type for abs(): 'str'
> 2023-02-09 13:25:04,544 [info] Couldn't calculate mutual_info_classif because of: bad operand type for abs(): 'str'
> 2023-02-09 13:25:05,445 [info] Couldn't calculate f_regression because of: bad operand type for abs(): 'str'
Liblinear failed to converge, increase the number of iterations.
> 2023-02-09 13:25:09,278 [info] votes needed to be selected: 2
> 2023-02-09 13:25:10,854 [info] wrote target: {'name': 'parquet', 'kind': 'parquet', 'path': 'v3io:///projects/fraud-demo-dani/FeatureStore/transactions-fraud-short/parquet/vectors/transactions-fraud-short-latest.parquet', 'status': 'ready', 'updated': '2023-02-09T13:25:10.854393+00:00', 'size': 620751, 'par

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
fraud-demo-dani,...78a7bdf1,0,Feb 09 13:25:03,completed,feature_extraction,v3io_user=danikind=jobowner=danimlrun/client_version=1.3.0-rc20mlrun/client_python_version=3.9.13host=feature-extraction-lpctf,df_artifact,k=18min_votes=2label_column=labeloutput_vector_name=transactions-fraud-shortignore_type_errors=True,top_features_vector=store://feature-vectors/fraud-demo-dani/transactions-fraud-short,chi2LinearSVCLogisticRegressionExtraTreesClassifierfeature_scoresmax_scaled_scores_feature_scoresselected_features_countselected_features


> 2023-02-09 13:25:14,269 [info] run executed, status=completed


In [10]:
mlrun.get_dataitem(feature_selection_run.outputs['top_features_vector']).as_df().tail(5)

,amount_max_2h,amount_min_2h,amount_avg_2h,es_transportation_sum_1d,es_health_sum_1d,es_otherservices_sum_1d,es_food_sum_1d,es_hotelservices_sum_1d,es_barsandrestaurants_sum_1d,es_tech_sum_1d,es_sportsandtoys_sum_1d,es_wellnessandbeauty_sum_1d,es_hyper_sum_1d,es_fashion_sum_1d,es_home_sum_1d,label
49995,48.59,13.52,31.870000,31,0,0,2,0,1,0,0,3,2,0,0,0
49996,42.45,1.97,20.648750,30,0,0,3,0,2,0,0,3,1,0,1,0
49997,40.40,18.36,31.190000,36,2,0,2,0,0,0,0,0,0,0,0,0
49998,77.76,23.21,47.590000,37,0,0,1,0,0,0,0,1,0,0,0,0
49999,26.74,11.77,19.636667,14,0,0,0,0,1,0,0,1,0,1,0,0


## Step 5 - Train our models with top features

Following the feature selection, we train new models using the resultant features. We can observe the accuracy and other results remain high
meaning we get a model that requires less features to be accurate and thus less error-prone.

In [11]:
# Defining our training task, including our feature vector, label and hyperparams definitions
ensemble_train_task = mlrun.new_task('training', 
                      inputs={'dataset': feature_selection_run.outputs['top_features_vector']},
                      params={'label_columns': 'label'}
                     )
ensemble_train_task.with_hyper_params(training_params, strategy='list', selector='max.accuracy')

classifier_fn.run(ensemble_train_task)

> 2023-02-09 13:25:14,375 [info] starting run training uid=1b9ad3c2ee764fe89695b1c8c8fe41c6 DB=http://mlrun-api:8080
> 2023-02-09 13:25:14,536 [info] Job is running in the background, pod: training-dlczb
> 2023-02-09 13:25:19,725 [info] test_set or train_test_split_size are not provided, setting train_test_split_size to 0.2
> 2023-02-09 13:25:20,791 [info] label columns: label
> 2023-02-09 13:25:20,791 [info] Sample set not given, using the whole training set as the sample set
> 2023-02-09 13:25:21,082 [info] training 'transaction_fraud_rf'
> 2023-02-09 13:25:27,402 [info] test_set or train_test_split_size are not provided, setting train_test_split_size to 0.2
> 2023-02-09 13:25:28,531 [info] label columns: label
> 2023-02-09 13:25:28,532 [info] Sample set not given, using the whole training set as the sample set
> 2023-02-09 13:25:28,545 [info] training 'transaction_fraud_xgboost'
> 2023-02-09 13:25:36,862 [info] test_set or train_test_split_size are not provided, setting train_test_s

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
fraud-demo-dani,...c8fe41c6,0,Feb 09 13:25:19,completed,training,v3io_user=danikind=jobowner=danimlrun/client_version=1.3.0-rc20mlrun/client_python_version=3.9.13,dataset,label_columns=label,best_iteration=3accuracy=0.993f1_score=0.4776119402985075precision_score=0.6274509803921569recall_score=0.3855421686746988,feature-importancetest_setconfusion-matrixroc-curvescalibration-curvemodeliteration_resultsparallel_coordinates


> 2023-02-09 13:25:45,330 [info] run executed, status=completed


## Done!

You've completed Part 2 of the model training with the feature store.
Proceed to [Part 3](03-deploy-serving-model.ipynb) to learn how to deploy and monitor the model.